First, let's import required packages. 

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Then, we read data and check first couple of rows as well as its size.

In [21]:
data = pd.read_csv('../data/raw/train.csv')

As the dataset is relatively large with 1306122 rows and we will have some difficulty if the whole dataset is considered. Therefore, we do random stratified sampling and keep only 20% of data.

In [22]:
data = data.groupby('target', group_keys=False).apply(lambda x: x.sample(frac=0.20))

In [23]:
data.shape

(261224, 3)

In [24]:
data.head()

,qid,question_text,target
222975,2b9d5a94f4745b82ba1b,What are some solid potato salad recipes?,0
223620,2bbb43ce48aebb236563,How do I stop blaming myself for my sociopathi...,0
420823,527ad768957a81887317,Will La La Land get a sequel?,0
268964,34a6a43acf8a1201f0b9,Why is North Korea a failed state?,0
1152269,e1c7f7d19bc2270a19b1,What are some locations of volcanoes?,0


In [25]:
X = data[['qid', 'question_text']]
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, 
                                                    test_size=0.25, random_state=42)

In [26]:
X_train.shape

(195918, 2)

In [27]:
X_test.shape

(65306, 2)

In [32]:
np.mean(y_train)

0.06187282434487898

In [33]:
np.mean(y_test)

0.06186261599240499